# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.shape

(21525, 12)

In [3]:
df.columns
#названия столбцов корректные

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Вывод**

Таблица содержит категориальные и количественные переменные, а также различные артефакты, требуется предобработка

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
df.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

пустые значения найдены в столбце с днями трудового стажа 
и в столбце с ежемесячным доходом,причем их одинаковое количество, в связи с этим можно 
сделать вывод, что это люди без опыта работы и без заработка

In [5]:
#Проверка на то есть ли люди без опыта работы(days_employed = 0)
df.loc[df['days_employed'] == 0]
#Людей без опыта работы не найдено, 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Заменим пустые значения нулями и исследуем таблицу

In [6]:
df = df.fillna(0)
no_employed = df.loc[df['days_employed'] == 0]
no_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0.0,65,среднее,1,гражданский брак,1,M,пенсионер,0,0.0,сыграть свадьбу
26,0,0.0,41,среднее,1,женат / замужем,0,M,госслужащий,0,0.0,образование
29,0,0.0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,0.0,строительство жилой недвижимости
41,0,0.0,50,среднее,1,женат / замужем,0,F,госслужащий,0,0.0,сделка с подержанным автомобилем
55,0,0.0,54,среднее,1,гражданский брак,1,F,пенсионер,1,0.0,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,0.0,47,Среднее,1,женат / замужем,0,M,компаньон,0,0.0,сделка с автомобилем
21495,1,0.0,50,среднее,1,гражданский брак,1,F,сотрудник,0,0.0,свадьба
21497,0,0.0,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,0.0,строительство недвижимости
21502,1,0.0,42,среднее,1,женат / замужем,0,F,сотрудник,0,0.0,строительство жилой недвижимости


In [7]:
#Проверка на тип занятости у людей с пустыми значениями в столбце опыта работы
profession = no_employed['income_type'].unique()


Это либо предприниматель и компаньон, которые работают на себя, поэтому трудовой стаж не начисляется
Либо пенсионеры, у них скорее всего по ошибке не указан опыт работы
Вообще для проекта не так важен трудовой стаж, так как не нужно искать от него зависимость
поэтому чтобы не потерять данные заполним пустой строкой

Обработка столбца с доходами

In [8]:
def salary(data, work):
    table = data.loc[data['income_type'] == work]
    median_salary = table['total_income'].median()
    data.loc[(data['income_type'] == work) & (data['total_income'] == 0),'total_income'] = median_salary
    return data

In [9]:
for i in profession:
    df = salary(df, i)


In [10]:
#Так как заменили пустые значения нулями(до этого нулей в столбце не было) проверяем, остались ли в столбце с дохдами нули
df.loc[df['total_income'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


**Вывод**

Пустые значения в столбце с опытом работы были заменены на нули, так как для нашего исследования опыт работы не играет большой роли, а в пустые значения столбца с зарплатой была вставлена медианная зарплата по группе. Вставка медианных значений в этом случае оказывает меньшее влияние на общность данных. Причной возникновения пропущенных значений в этих столбцах может быть нежелание людей указывать опыть работы, либо его отсутствие, пустые значения в столбце с зарплатой можно объяснить ошибкой при сборе данных, ведь маловероятно, что люди без дохода стали бы брать кредит.

In [11]:
#таблица с данными на текущем этапе
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Замена типа данных

In [12]:
#перед заменой типа данных обработаем аретефакты
df['days_employed'] = abs(df['days_employed'])
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [13]:
#Проверка на то, есть ли отрицательные доходы
df.loc[df['total_income'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [14]:
#Проверка по опыту работы
work_experience_table = df
work_experience_table['years_employed'] = work_experience_table['days_employed'] / 365
work_experience_table

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.116912
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.026860
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.406637
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.300677
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,12.409087
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,942.294258
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,5.789991
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,8.527347


**Вывод**

Отрицательные значения были заменены на их абсолютную величину, так как возможно, что ошибки появились из-за неверного сбора данных

In [15]:
work_experience_table.loc[work_experience_table['years_employed'] > work_experience_table['dob_years']]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.235814
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля,1096.660649
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью,927.539597
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости,996.023258
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью,919.401832
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу,928.506483
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость,1058.897847
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью,992.222066
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем,1024.645783


Обнаружены артефакты в опыте работы, необыкновенно большое количество часов работы, но этот артефакт не требует обработки, так как
Это не влияет на проект

In [16]:
a = []
for i in df.columns:
    if i != 'years_employed':
        a.append(i)    
df = df.loc[:, a]
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [17]:
#замена типов данных
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


Вывод: Типы данных заменены в столбцах days_employed и total_income, так как целые числа более читаемые и с ними удобнее работать

### Обработка дубликатов

In [18]:
df.duplicated().sum()


54

In [19]:
#возможно дубликаты образовались в связи с заменой значений
#их количество не очень большое, поэтому можно просто избавиться от явных дубликатов
df = df.drop_duplicates()

In [20]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [21]:
#В столбце education видна разница в регистре
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [22]:
def low(string):
    return string.lower()
    

In [23]:
df['education'] = df['education'].apply(low)
df['education'].unique()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [24]:
df['family_status'].value_counts()
#Проверка на проблемы с регистром в столбце с инфой о семейном положении

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [25]:
df['children'].unique()
#Найдена ошибка в столбце с детьми, отрицательное кол-во детей и слишком большое количество

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [26]:
df['children'] = abs(df['children'])
df['children'].unique()
#обработка отрицательных значений

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([ 1,  0,  3,  2,  4, 20,  5])

In [27]:
#исследование большого количества детей
df.loc[df['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2941,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,1240,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524,свой автомобиль
21325,20,601,37,среднее,1,женат / замужем,0,F,компаньон,0,102986,профильное образование
21390,20,0,53,среднее,1,женат / замужем,0,M,компаньон,0,162401,покупка жилой недвижимости
21404,20,494,52,среднее,1,женат / замужем,0,M,компаньон,0,156629,операции со своей недвижимостью


In [28]:
#У 76 людей 20 детей, скорее всего была ошибка в сборе данных и детей двое
df.loc[df['children'] == 20,'children'] = 2
df['children'].unique()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


array([1, 0, 3, 2, 4, 5])

**Вывод**

обработан столбец с данными об образовании, так же удалены дубликаты

Возможно, дубликаты появились, в связи с заменой пустых значений в столбце с доходом, а также с совпадающими целями кредита
Еще причиной дубликатов может быть ошибка сбора данных

In [29]:
#текущий вид таблицы
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Лемматизация

In [30]:
from pymystem3 import Mystem
m = Mystem()


In [31]:
df.loc[:,'purpose_lemmas'] = df.loc[:,'purpose'].apply(m.lemmatize)
df

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


**Вывод**

Лемматизация проводилась при помощи библиотеки pymystem3, слова были заменены на их начальную форму. Выбор такого способа обработки слов был обусловлен тем, что они проще обрабатываются при категоризации

### Категоризация данных

In [32]:
def children_group(a):
    if a < 3:
        return 'стандартные семьи'
    else:
        return 'многодетные семьи'

In [33]:
df['children_group'] = df['children'].apply(children_group)
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,children_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",стандартные семьи
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",стандартные семьи
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",стандартные семьи
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",многодетные семьи
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",стандартные семьи
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",стандартные семьи
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",стандартные семьи
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",стандартные семьи
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",многодетные семьи


Проверим, для чего люди берут кредиты

In [34]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Рассматриваем, как выглядят леммы

In [35]:
df['purpose_lemmas'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             793
[на,  , проведение,  , свадьба, \n]                       773
[сыграть,  , свадьба, \n]                                 769
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           662
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 652
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            625
[покупка

Готовим столбец с группировкой по целям

In [36]:
def group_by_purpose(sheet):
    if 'свадьба' in sheet or ' свадьба' in sheet:
        return 'свадебный кредит'
    elif 'коммерческий' in sheet or ' коммерческий' in sheet:
        return 'кредит на коммерческую недвижимость'
    elif ('недвижимость' in sheet or 'жилье' in sheet) or (' недвижимость' in sheet or ' жилье' in sheet):
        return 'кредит на жилье'
    elif 'автомобиль' in sheet or ' автомобиль' in sheet:
        return 'автокредит'
    elif 'образование' in sheet or ' образование' in sheet:
        return 'кредит на образование'

In [37]:
df['purpose_groups'] = df['purpose_lemmas'].apply(group_by_purpose)
df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,children_group,purpose_groups
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",стандартные семьи,кредит на жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",стандартные семьи,автокредит
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",стандартные семьи,кредит на жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",многодетные семьи,кредит на образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",стандартные семьи,свадебный кредит
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",стандартные семьи,кредит на жилье
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",стандартные семьи,автокредит
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",стандартные семьи,кредит на жилье
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",многодетные семьи,автокредит


In [38]:
#удаляем ненужный столбец
df = df.drop('purpose_lemmas', axis=1)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_groups
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,стандартные семьи,кредит на жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,стандартные семьи,автокредит
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,стандартные семьи,кредит на жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетные семьи,кредит на образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,стандартные семьи,свадебный кредит
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,стандартные семьи,кредит на жилье
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,стандартные семьи,автокредит
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,стандартные семьи,кредит на жилье
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетные семьи,автокредит


In [39]:
def income_grouped(summa):
    if summa < 50000:
        group = 'малоимущие'
    elif summa < 100000:
        group = 'средний доход'
    elif summa < 120000:
        group = 'доход выше среднего'
    else:
        group = 'высокий доход'
    return group

In [40]:
df['income_grouped'] = df['total_income'].apply(income_grouped)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_groups,income_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,стандартные семьи,кредит на жилье,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,стандартные семьи,автокредит,доход выше среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,стандартные семьи,кредит на жилье,высокий доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетные семьи,кредит на образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,стандартные семьи,свадебный кредит,высокий доход
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,стандартные семьи,кредит на жилье,высокий доход
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,стандартные семьи,автокредит,высокий доход
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,стандартные семьи,кредит на жилье,средний доход
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетные семьи,автокредит,высокий доход


In [41]:
df['purpose_groups'].unique()

array(['кредит на жилье', 'автокредит', 'кредит на образование',
       'свадебный кредит', 'кредит на коммерческую недвижимость'],
      dtype=object)

In [42]:
def purpose_id(string):
    if string == 'кредит на жилье':
        ident = 0
    elif string == 'автокредит':
        ident = 1
    elif string == 'кредит на образование':
        ident = 2
    elif string == 'свадебный кредит':
        ident = 3
    else:
        ident = 4
    return ident
        

In [43]:
df['purpose_id'] = df['purpose_groups'].apply(purpose_id)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_groups,income_grouped,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,стандартные семьи,автокредит,доход выше среднего,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетные семьи,кредит на образование,высокий доход,2
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,стандартные семьи,свадебный кредит,высокий доход,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,стандартные семьи,кредит на жилье,высокий доход,0
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,стандартные семьи,автокредит,высокий доход,1
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,стандартные семьи,кредит на жилье,средний доход,0
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетные семьи,автокредит,высокий доход,1


In [44]:
purpose_dict = df.loc[:,['purpose_id', 'purpose_groups']]
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
purpose_dict

,purpose_id,purpose_groups
0,0,кредит на жилье
1,1,автокредит
2,2,кредит на образование
3,3,свадебный кредит
4,4,кредит на коммерческую недвижимость


In [45]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [46]:
def education_id(string):
    if string == 'высшее':
        ident = 1
    elif string == 'среднее':
        ident = 2
    elif string == 'неоконченное высшее':
        ident = 3
    elif string == 'начальное':
        ident = 4
    elif string == 'ученая степень':
        ident = 5
    return ident

In [47]:
df['education_id'] = df['education'].apply(education_id)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_groups,income_grouped,purpose_id
0,1,8437,42,высшее,1,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0
1,1,4024,36,среднее,2,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,стандартные семьи,автокредит,доход выше среднего,1
2,0,5623,33,среднее,2,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0
3,3,4124,32,среднее,2,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетные семьи,кредит на образование,высокий доход,2
4,0,340266,53,среднее,2,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,стандартные семьи,свадебный кредит,высокий доход,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,2,гражданский брак,1,F,компаньон,0,224791,операции с жильем,стандартные семьи,кредит на жилье,высокий доход,0
21521,0,343937,67,среднее,2,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,стандартные семьи,автокредит,высокий доход,1
21522,1,2113,38,среднее,2,гражданский брак,1,M,сотрудник,1,89672,недвижимость,стандартные семьи,кредит на жилье,средний доход,0
21523,3,3112,38,среднее,2,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетные семьи,автокредит,высокий доход,1


In [48]:
education_dict = df.loc[:, ['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,1
1,среднее,2
2,неоконченное высшее,3
3,начальное,4
4,ученая степень,5


In [49]:
df['income_grouped'].unique()

array(['высокий доход', 'доход выше среднего', 'средний доход',
       'малоимущие'], dtype=object)

In [50]:
def income_id(string):
    if string == 'высокий доход':
        ident = 0
    elif string == 'доход выше среднего':
        ident = 1
    elif string == 'средний доход':
        ident = 2
    else:
        ident = 3
    return ident

In [51]:
df['income_id'] = df['income_grouped'].apply(income_id)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_groups,income_grouped,purpose_id,income_id
0,1,8437,42,высшее,1,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0,0
1,1,4024,36,среднее,2,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,стандартные семьи,автокредит,доход выше среднего,1,1
2,0,5623,33,среднее,2,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0,0
3,3,4124,32,среднее,2,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетные семьи,кредит на образование,высокий доход,2,0
4,0,340266,53,среднее,2,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,стандартные семьи,свадебный кредит,высокий доход,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,2,гражданский брак,1,F,компаньон,0,224791,операции с жильем,стандартные семьи,кредит на жилье,высокий доход,0,0
21521,0,343937,67,среднее,2,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,стандартные семьи,автокредит,высокий доход,1,0
21522,1,2113,38,среднее,2,гражданский брак,1,M,сотрудник,1,89672,недвижимость,стандартные семьи,кредит на жилье,средний доход,0,2
21523,3,3112,38,среднее,2,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетные семьи,автокредит,высокий доход,1,0


In [52]:
income_dict = df.loc[:,['income_id', 'income_grouped']]
income_dict = income_dict.drop_duplicates().reset_index(drop=True)
income_dict

,income_id,income_grouped
0,0,высокий доход
1,1,доход выше среднего
2,2,средний доход
3,3,малоимущие


In [53]:
df['children_group'].unique()

array(['стандартные семьи', 'многодетные семьи'], dtype=object)

In [54]:
def children_id(string):
    if string == 'стандартные семьи':
        ident = 0
    else:
        ident = 1
    return ident

In [55]:
df['children_group_id'] = df['children_group'].apply(children_id)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_group,purpose_groups,income_grouped,purpose_id,income_id,children_group_id
0,1,8437,42,высшее,1,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0,0,0
1,1,4024,36,среднее,2,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,стандартные семьи,автокредит,доход выше среднего,1,1,0
2,0,5623,33,среднее,2,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,стандартные семьи,кредит на жилье,высокий доход,0,0,0
3,3,4124,32,среднее,2,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,многодетные семьи,кредит на образование,высокий доход,2,0,1
4,0,340266,53,среднее,2,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,стандартные семьи,свадебный кредит,высокий доход,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,2,гражданский брак,1,F,компаньон,0,224791,операции с жильем,стандартные семьи,кредит на жилье,высокий доход,0,0,0
21521,0,343937,67,среднее,2,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,стандартные семьи,автокредит,высокий доход,1,0,0
21522,1,2113,38,среднее,2,гражданский брак,1,M,сотрудник,1,89672,недвижимость,стандартные семьи,кредит на жилье,средний доход,0,2,0
21523,3,3112,38,среднее,2,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,многодетные семьи,автокредит,высокий доход,1,0,1


In [56]:
children_group_dict = df[['children_group_id', 'children_group']]
children_group_dict = children_group_dict.drop_duplicates().reset_index(drop=True)
children_group_dict

,children_group_id,children_group
0,0,стандартные семьи
1,1,многодетные семьи


In [57]:
family_group_dict = df.loc[:, ['family_status_id', 'family_status']]
family_group_dict = family_group_dict.drop_duplicates().reset_index(drop=True)
family_group_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [58]:
# КОД РЕВЬЮЕРА

dict(zip(df['family_status_id'], df['family_status']))

{0: 'женат / замужем',
 1: 'гражданский брак',
 2: 'вдовец / вдова',
 3: 'в разводе',
 4: 'Не женат / не замужем'}

**Вывод**

Проведена категоризация по целям кредита, семейному положению и уровню дохода, это было сделано для того чтобы ответить на вопросы  проекта

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [59]:


df.groupby(['children_group', 'children']).agg({'debt':['sum', 'count', lambda x: '{:.2%} '.format(x.mean())]})

debt                  
                             sum  count <lambda_0>
children_group    children                        
многодетные семьи 3           27    330     8.18% 
                  4            4     41     9.76% 
                  5            0      9     0.00% 
стандартные семьи 0         1063  14107     7.54% 
                  1          445   4856     9.16% 
                  2          202   2128     9.49%

In [60]:
def debstors_percent(a):
    return '{:.2%} '.format(a.mean())

In [61]:
children_group = df.groupby(['children_group_id', 'children'])['debt'].agg(['sum', 'count', debstors_percent])
children_group.reset_index().replace({'children_group_id': children_group_dict.children_group.to_dict()})

,children_group_id,children,sum,count,debstors_percent
0,стандартные семьи,0,1063,14107,7.54%
1,стандартные семьи,1,445,4856,9.16%
2,стандартные семьи,2,202,2128,9.49%
3,многодетные семьи,3,27,330,8.18%
4,многодетные семьи,4,4,41,9.76%
5,многодетные семьи,5,0,9,0.00%


**Вывод**

По полученной таблице можно сделать вывод, что чаще всего возвращают кредиты вдовцы, реже холостяки и люди, проживающие в гражданском браке

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [63]:
grouped_by_income = df.groupby('income_id').agg({'debt':['sum', 'count', debstors_percent]})
grouped_by_income.reset_index().replace({'income_id': income_dict.income_grouped.to_dict()})

income_id  debt                        
                         sum  count debstors_percent
0        высокий доход  1155  14232           8.12% 
1  доход выше среднего   232   2776           8.36% 
2        средний доход   331   4091           8.09% 
3           малоимущие    23    372           6.18%

Данные показывают, что чаще всего отдают кредиты люди с небольшим заработком, это может говорить о том, что они берут деньги на что-нибудь необходимое, поэтому всегда возвращают, реже всего платят по кредитам люди с доходами выше среднего, возможно из-за того, что они переоценивают свои финансовые возможности

- Как разные цели кредита влияют на его возврат в срок?

In [64]:
group_by_purpose = df.groupby('purpose_id').agg({'debt':['sum', 'count', debstors_percent]})
group_by_purpose.reset_index().replace({'purpose_id': purpose_dict.purpose_groups.to_dict()})

purpose_id debt                       
                                        sum count debstors_percent
0                      кредит на жилье  683  9502           7.19% 
1                           автокредит  403  4308           9.35% 
2                кредит на образование  370  4014           9.22% 
3                     свадебный кредит  186  2335           7.97% 
4  кредит на коммерческую недвижимость   99  1312           7.55%

**Вывод**

Чаще всего люди отдают кредиты на жилье, реже кредиты на машину, причиной может стать то, что машины чаще покупают более молодые и менее платежеспособные люди

## Шаг 4. Общий вывод

<font color='blue'> Полученный результат говорит о том, что большинство людей выплачивают кредиты, однако из тех кто не выплачивает кредиты
можно выделить группы риска по займу.
Это холостяки - почти 10% не платят по кредиту, представители среднего класса-8.62% не платят по кредиту, а также люди, оформляющие автокредит-почти 10% не платят по кредиту. 
И, наоборот, к надежным заемщикам можно отнести
    семьи с детьми - 7,54% невозврата
    Вдовцы и вдовы - 6,57% невозврата
    малоимущие - 6.18% невозврата
    Люди, берущие кредит на жилье - 7.19% невозврата
</font>